# Figure 4

### TODO: make sure code works

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from src.theory import fixed_point
from src.theory import c_map
from src.theory import c_map_slope

In [11]:
rates = np.linspace(0.1, 1, 100)
mu2s = 1/rates
fps = []
fp_slopes = []
for p in rates:
    mu2 = 1/p
    fpoint = fixed_point(c_map, p, p*2, mu2)
    fps.append(fpoint)

    slope = c_map_slope(fpoint, p*2)
    fp_slopes.append(slope)

fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 5))
pickle_in = open("src/results/under_overflow.pk","rb")
example_dict = pickle.load(pickle_in)[:120]
p = 0.6
num_layers = 1000
nets = np.linspace(10, num_layers, 12, dtype=int)[:10]
inits = np.round(np.linspace(0.1, 2.5, 12),1)
xv, yv = np.meshgrid(nets, inits, sparse=False, indexing='ij')

Z1 = np.log(np.array(example_dict))

bad_indices = np.isnan(Z1) + np.isinf(Z1)
Z1 = np.ma.array(Z1, mask=bad_indices)
cmap = matplotlib.cm.get_cmap(name="Spectral_r")
cmap.set_bad('white', 1.)

pcm = ax1.pcolormesh(yv-0.1, xv-0.1, Z1.reshape(10,12), cmap=cmap)
cbar = fig.colorbar(pcm, ax=ax1, extend='max')
cbar.ax.set_title('$log(q^l)$')

ax1.set_xlabel('Weight initialisation ($\sigma^2_w$)')

ax1.set_ylabel("Number of layers")
ax1.set_title("Variance propagation depth:\ndropout with $p$ = 0.6,\ncrit. init. at $\sigma^2_w = 1.2$")

max_depth = 0
init_theory = np.linspace(0, 2.4, 1000)
depth_per_p_theory = depth("Dropout", init_theory, p)
max_depth = np.max([max_depth, np.max(depth_per_p_theory)])
ax1.plot(init_theory, depth_per_p_theory, label="Theory", c='cyan', linewidth=3)

# plot critical point
crit_point = critical_point("Dropout", p)
ax1.plot([crit_point,]*2, [0, num_layers], color="black", linestyle="--", label="criticality")

ax1.set_ylim(0, 820)
ax1.legend()
ax1.set_xticks(inits[:-1:2])
ax1.text(0.1, 400, 'Underflow', fontsize=10)
ax1.text(1.7, 400, 'Overflow', fontsize=10)

pickle_in = open("src/results/val_loss_per_depth.pk","rb")
example_dict = np.array(pickle.load(pickle_in))[:,1]
rates = np.linspace(0.1, 1, 100)

nets = np.linspace(2, 40, 10, dtype=int)
d_rates = 2*np.linspace(0.1, 1, 10)
xv, yv = np.meshgrid(nets, d_rates, sparse=False, indexing='ij')

Z1 = np.array(example_dict)
pcm = ax2.pcolormesh(yv+0.1, xv+0.1, Z1.reshape(10,10), cmap='Spectral_r')
cbar1 = fig.colorbar(pcm, ax=ax2, extend='max')
cbar1.ax.set_title('Val. loss')

# Now do a second plot coloring the curve using a continuous colormap
t = rates #np.linspace(0, 10, 200)
x = 2*rates
y = 6*depth_scale(fp_slopes)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

lc = LineCollection(segments, cmap=plt.get_cmap('Greens'))
lc.set_array(t)
lc.set_linewidth(3)
cbar2 = fig.colorbar(lc, ax=ax2)
cbar2.ax.set_title('p')

ax2.add_collection(lc)
ax2.set_xlabel("Critical initialisation for $p$ ($\sigma^2_w$)")
ax2.set_ylabel("Number of layers")
ax2.text(1.5, 19, r'$6\xi_c$', fontsize=12, color="white")
ax2.set_title("Depth at criticality")
ax2.set_xlim(0.3, 2.1)
ax2.set_ylim(2, 40)
# ax2.legend(loc=(0.1, 0.65))
# plt.show()
plt.gcf().tight_layout()
plt.savefig("depth_scales.pdf")

FileNotFoundError: [Errno 2] No such file or directory: 'src/results/under_overflow.pk'